# Compute solution for sinusoidal problem with extreme compression

This jupyter notebook computes the solution of the Transport equation for some initial conditions ρ0 and u0 using the DG, DG +-limiter or DFRG methods.

Problem parameters are specified and saved in the struct FRGSemidiscretizationDGMultiD.

The semidiscretization struct is passed onto a custom time-stepper implementation, in this case a 3rd order SSPRK.

The solution is visualized using GLMakie with a time slider to adjust the time point. Plotted points are evaluated based on the Galerkin solution, and the number of points plotted can be adjusted.

Exampled are saved in a .ser file format as a vector [ sd, xs, ρ_out, u0, t_out]. This includes the sermidescretization (sd) , points specifying Lagrange polynomials (xs), the density solution over time (ρ_out), the velocity profile (u0), and the evaluated time points (t_out).

Solutions can be saved and extracted from .ser files as needed.


In [ ]:
using FRG
using LinearAlgebra
using GLMakie
using Serialization

Specify problem parameters

In [ ]:
d = 1
L = [ 1.0 ] # domain length
m = [ 32 ] # number of discretization points
p = [ 1 ] # polynomial order for basis functions
n_q = [ 5  ] # number of quadrature points
Δx = L ./ m # size of cell

FR_weight = true # switch from DG to DFRG when true
DG_limiter = false # switch from DG to DG+limiter when true
catch_error = false # returns solution until method fails in case of failure

n_plot = [ 500 ]
Δx_plot = L./( n_plot .- 1 )

xs = FRG.initialize_spacial_array(d,m,p,Δx)
x_plot = FRG.initialize_plot_array(n_plot,Δx_plot,L) |> collect

Δt = 1 / maximum(m) / 1.6 / 10 # time step 
Δt_record = 0.1 # time between recorded points
T = 100.0 # time

# initial conditions of length m*p (number of parameters)
ρ_fxn(x) = 1.0
u_fxn(x) = [ sin(2*pi*x[1]) .+ 1.01  ]

ρ0 = map( x-> ρ_fxn(x) , xs )
u0 = map( x -> u_fxn(x) , xs );

Initialize semidiscretization

In [ ]:
sd = FRG.FRGSemidiscretizationDGMultiD( p,
                                        d,
                                        Δx,
                                        Δt,
                                        m,
                                        n_q,
                                        f_type="upwind",
                                        FR_weight=FR_weight,
                                        quad_type="clenshaw_curtis" );

Solve the problem

In [ ]:
# solve the problem

ρ_out , t_out = FRG.rk3(    sd, 
                            T, 
                            ρ0, 
                            u0, 
                            Δt_record, 
                            DG_limiter=DG_limiter,  
                            catch_error=catch_error );

Plot the solution

In [ ]:
n_plot = [ 300 ]  # specify number of points to plot
Δx_plot = L./( n_plot .- 1 ) 
x_plot = collect( FRG.initialize_plot_array(n_plot,Δx_plot,L) )

ρ_plot, u_plot = FRG.evaluate_plot_points(  sd,
                                            x_plot,
                                            ρ_out,
                                            u0,
                                            t_out   )

fig = FRG.plot_1D_FRG_solution(     sd, 
                                    ρ_plot, 
                                    u_plot, 
                                    x_plot, 
                                    t_out   )

display(fig)

Save computed solution

In [ ]:
#=
file_name = FRG.generate_saved_example_file_name(   sd::FRGSemidiscretizationDGMultiD,
                                                    "bump",
                                                    DG_limiter,
                                                    T)

file_folder = "saved examples" 
directory_name = dirname(@__DIR__)
file_path = joinpath(directory_name, file_folder, file_name)  

if !isfile(file_path)
    open(file_path, "w") do f
        serialize(f, [ sd, xs, ρ_out, u0, t_out] )
    end
end
=#

Open saved example

In [ ]:
#=
file_name = "DFRG_bump_1D_8_p_1_q_11_T_50-0.ser"
file_folder = "saved examples/1D_bump" 
directory_name = dirname(@__DIR__)
file_path = joinpath(directory_name, file_folder, file_name)  

sd, xs, ρ_out, u0, t_out = open(file_path, "r") do file
    deserialize(file)
end;
=#